In [ ]:
import cv2
import numpy as np
import tensorflow as tf

In [ ]:

# Constants
IMG_WIDTH = 30
IMG_HEIGHT = 30


In [ ]:
# Functions
def get_contours(image):
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])
    return contours


def sort_contours(contours):
    base_rect=[]
    w=int(IMG_WIDTH)
    h=int(IMG_HEIGHT)
    for c in contours :
        x,y,w,h= cv2.boundingRect(c)
        rect=[x,y,w,h]
        base_rect.append(rect)
    is_true_rect=[]
    for r in base_rect:
        l=[]
        for rec in base_rect:
            flag=0
            if rec!=r:
                if r[0]<(rec[0]+rec[2]+10) and rec[0]<(r[0]+r[2]+10) and r[1]<(rec[1]+rec[3]+10) and rec[1]<(r[1]+r[3]+10):
                    flag=1
                l.append(flag)
            if rec==r:
                l.append(0)
        is_true_rect.append(l)
    shit_rect=[]
    for i in range(0,len(contours)):
        for j in range(0,len(contours)):
            if is_true_rect[i][j]==1:
                a1=base_rect[i][2]*base_rect[i][3]
                a2=base_rect[j][2]*base_rect[j][3]
                if(a1==min(a1,a2)):
                    shit_rect.append(base_rect[i])
    return [i for i in base_rect if i not in shit_rect]


def extract_digits(image, groups):
    train_data = []
    for r in groups:
        x=r[0]
        y=r[1]
        w=r[2]
        h=r[3]
        im_crop =image[y:y+h,x:x+w]
        
        im_resize = cv2.resize(im_crop,(IMG_WIDTH,IMG_HEIGHT))
        cv2.imshow("work",im_resize)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        im_resize=np.reshape(im_resize,(1,900))
        train_data.append(im_resize)
    return train_data


def predict_image(image_path, model):
    # Load the image and preprocess it
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image=~image
    image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)[1]
    contours = get_contours(image)
    groups = sort_contours(contours)
    digits = extract_digits(image, groups)
    result = []
    for d in digits:
        d=np.array(d)
        # d=d.reshape(1,1,IMG_WIDTH,IMG_HEIGHT)
        result.append(model.predict(d))
    # Convert the predicted classes to digits
    digits = [np.argmax(r) for r in result]
    return digits


In [5]:
# Load the model
model = tf.keras.models.load_model('handwritten_equation_solver.h5')

# Predict an example image
digits = predict_image('test2.jpeg', model)

exp = ''
for d in digits:
    if d < 10:
        exp += (str(d))
    elif d == 10:
        exp += ('+')
    elif d == 11:
        exp += ('-')
    else:
        exp += ('*')

result = eval(exp)

print(f"{exp} = {result}")

1/1 [==============================] - 0s 23ms/step
7+3 = 10
